In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
TFLite 모델 서명 실행기
- TFLite SavedModel 서명(infer, save) 실행
- CSV 테스트 데이터 로드 및 예측

Usage:
    python tflite_runner.py \
        --model_path /path/multi_cnn_sign.tflite \
        --x_csv /path/test_x.csv \
        --save_ckpt /path/model.ckpt \
        [--num_samples 1]
"""
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

def load_interpreter(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter


def run_inference(interpreter, x_csv, num_samples):
    df = pd.read_csv(x_csv, header=None)
    arr = df.to_numpy(dtype=np.float32)
    # reshape to [samples, time, features]
    arr = arr.reshape(-1, 300, 12)
    arr = arr[:num_samples]
    # split sensors: [4, batch, 300,3]
    sensors = [arr[:, :, i*3:(i+1)*3] for i in range(4)]
    stack = np.stack(sensors, axis=0)
    infer = interpreter.get_signature_runner("infer")
    out = infer(x=stack)
    return out


def save_checkpoint(interpreter, ckpt_path):
    save = interpreter.get_signature_runner("save")
    save(checkpoint_path=np.array(ckpt_path, dtype=np.string_))
    print(f"Checkpoint saved to {ckpt_path}")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True, help='Path to .tflite model')
    parser.add_argument('--x_csv', required=True, help='Test CSV path')
    parser.add_argument('--save_ckpt', required=True, help='Checkpoint path to save')
    parser.add_argument('--num_samples', type=int, default=1,
                        help='Number of samples to infer')
    args = parser.parse_args()

    interpreter = load_interpreter(args.model_path)
    results = run_inference(interpreter, args.x_csv, args.num_samples)
    print("Inference results:", results['logits'])
    save_checkpoint(interpreter, args.save_ckpt)

if __name__ == '__main__':
    main()

2024-08-07 13:36:00.418072: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 13:36:00.441005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 13:36:00.769887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO: Created TensorFlow Lite delegate for select TF ops.
2024-08-07 13:36:01.219827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node